In [10]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Defaul')

In [11]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [12]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/28 17:58:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/28 17:58:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [1]:
def minTemperature(df,spark):
    return spark.sql("SELECT min(temperature) as mintemp from washing").first().mintemp

In [2]:
def meanTemperature(df,spark):
    return spark.sql("SELECT avg(temperature) as meantemp from washing").first().meantemp

In [3]:
def maxTemperature(df,spark):
    return spark.sql("SELECT max(temperature) as maxtemp from washing").first().maxtemp

In [4]:
def sdTemperature(df,spark):
    return spark.sql("SELECT stddev(temperature) as stddevtemp from washing").first().stddevtemp

In [5]:
def skewTemperature(df,spark): 
    return spark.sql("""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature-%s,3)/POWER(%s,3)
    )

as skewTemp from washing
                    """ %(meanTemperature(df,spark),sdTemperature(df,spark))).first().skewTemp

In [6]:
def kurtosisTemperature(df,spark): 
    return spark.sql("""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature-%s,4)/POWER(%s,4)
    )

as kurtTemp from washing
                    """ %(meanTemperature(df,spark),sdTemperature(df,spark))).first().kurtTemp

In [7]:
def correlationTemperatureHardness(df,spark):
    return df.stat.corr("temperature", "hardness")

In [8]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

--2022-05-28 17:57:10--  https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/claimed/blob/master/coursera_ds/washing.parquet?raw=true [following]
--2022-05-28 17:57:10--  https://github.com/IBM/claimed/blob/master/coursera_ds/washing.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/claimed/raw/master/coursera_ds/washing.parquet [following]
--2022-05-28 17:57:10--  https://github.com/IBM/claimed/raw/master/coursera_ds/washing.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/claimed/master/coursera_ds/washing.parquet [following]
--2022-05-28 17:57:10

In [13]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

In [14]:
min_temperature = 0
mean_temperature = 0
max_temperature = 0
sd_temperature = 0
skew_temperature = 0
kurtosis_temperature = 0
correlation_temperature = 0

In [21]:
min_temperature = minTemperature(df,spark)
print(min_temperature)


80


In [22]:
mean_temperature = meanTemperature(df,spark)
print(mean_temperature)


90.03800298062593


In [23]:
max_temperature = maxTemperature(df,spark)
print(max_temperature)


100


In [24]:
sd_temperature = sdTemperature(df,spark)
print(sd_temperature)


6.1007610586219725


In [25]:
skew_temperature = skewTemperature(df,spark)
print(skew_temperature)


0.010398374583336334


In [26]:
kurtosis_temperature = kurtosisTemperature(df,spark)
print(kurtosis_temperature)


1.773427150876985


In [ ]:
correlation_temperature = correlationTemperatureHardness(df,spark)
print(correlation_temperature)

In [27]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2022-05-28 18:00:48--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-05-28 18:00:48 (16.5 MB/s) - ‘rklib.py’ saved [2540/2540]



In [29]:
from rklib import submitAll
import json

key = "Suy4biHNEeimFQ479R3GjA"
email = "mido99ahmed14@gmail.com"
token = "YGylC8DNEJsrEi9f" #you can obtain it from the grader page on Coursera

In [30]:
parts_data = {}
parts_data["FWMEL"] = json.dumps(min_temperature)
parts_data["3n3TK"] = json.dumps(mean_temperature)
parts_data["KD3By"] = json.dumps(max_temperature)
parts_data["06Zie"] = json.dumps(sd_temperature)
parts_data["Qc8bI"] = json.dumps(skew_temperature)
parts_data["LoqQi"] = json.dumps(kurtosis_temperature)
parts_data["ehNGV"] = json.dumps(correlation_temperature)



submitAll(email, token, key, parts_data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"TzU1P","id":"sUpST4RAEeawAApvKZgcCQ~TzU1P~OaERaN6wEey5rQ67SoeUzw","courseId":"sUpST4RAEeawAApvKZgcCQ"}],"paging":{},"linked":{}}
-------------------------
